Text Mining : TF-IDF and Cosine Similarity from Scratch

Table of Contents:

1. Term Frequency (TF)
2. Inverse Document Frequency (IDF)
3. TF * IDF
4. Vector Space Models and Representation – Cosine Similarity

*** Any feedback or feature requests are welcome!***


Let us imagine that you are doing a search on below documents with the following query: **life learning**

* **Document 1** : I want to start learning to charge something in life.
* **Document 2** : learning something about me no one else knows
* **Document 3** : Never stop learning

The query is a free text query. It means a query in which the terms of the query are typed freeform into the search interface, without any connecting search operators.

Let us go over each step in detail to see how it all works.

# 1.Term Frequency(TF)
Term Frequency also known as TF measures the number of times a term (word) occurs in a document. Given below is the code and the terms and their frequency on each of the document.


In [46]:
import math
import pandas as pd
import numpy as np

In [47]:
# Load dataset
path = '/content/drive/MyDrive/Colab Notebooks/CSC 820/transcripts.csv'
data = pd.read_csv(path)

In [48]:
# Adjust sample size depending on amount of time you have to run program
reduced_data = data.sample(frac=0.01, random_state=42)
documents = reduced_data['transcript'].tolist()
# Query strings
query1 = "life learning"
query2 = "hard work"
query3 = "self actualization"

**NOTE :** Text Preprocessing Steps are ignored as the objective of this kernel is to explain and develop TF-IDF and cosine similarity from scratch

In [49]:
#Normalized Term Frequency
def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

# Compute term frequency for a list of documents
def compute_tf(docs_list):
    for doc in docs_list:
        word_list = doc.split()
        word_dict = dict.fromkeys(set(word_list), 0)
        for word in word_list:
            word_dict[word] += 1
        df = pd.DataFrame([word_dict])
        print(df)

* In reality each document will be of different size. On a large document the frequency of the terms will be much higher than the smaller ones. Hence we need to normalize the document based on its size.
* A simple trick is to divide the term frequency by the total number of terms.
* For example in Document 1 the term game occurs two times. The total number of terms in the document is 10. Hence the normalized term frequency is 2 / 10 = 0.2.


Given below are the normalized term frequency for all the documents.

In [50]:
# Compute normalized term frequency for a list of documents
def compute_normalizedtf(documents):
    tf_doc = []
    for txt in documents:
        sentence = txt.split()
        norm_tf = dict.fromkeys(set(sentence), 0)
        for word in sentence:
            norm_tf[word] = termFrequency(word, txt)
        tf_doc.append(norm_tf)
    return tf_doc

tf_doc = compute_normalizedtf(documents)

# 2. Inverse Document Frequency (IDF)

* The main purpose of doing a search is to find out relevant documents matching the query.
* In Term Frequecy all terms are considered equally important. In fact certain terms that occur too frequently have little power in determining the relevance.
* We need a way to weigh down the effects of too frequently occurring terms. Also the terms that occur less in the document can be more relevant.
* We need a way to weigh up the effects of less frequently occurring terms. Logarithms helps us to solve this problem.Logarithms helps us to solve this problem.


Let us compute IDF for the term start

IDF(start) = 1 + loge(Total Number Of Documents / Number Of Documents with term start in it)

There are 3 documents in all = Document1, Document2, Document3
The term start appears in Document1

 IDF(start) = 1 + loge(3 / 1)
            = 1 + 1.098726209
            = 2.098726209

In [51]:
# Compute inverse document frequency for all documents
def inverseDocumentFrequency(term, allDocuments):
    numDocumentsWithThisTerm = 0
    for doc in allDocuments:
        if term.lower() in doc.lower().split():
            numDocumentsWithThisTerm += 1
    if numDocumentsWithThisTerm > 0:
        return 1.0 + math.log(float(len(allDocuments)) / numDocumentsWithThisTerm)
    else:
        return 1.0

def compute_idf(documents):
    idf_dict = {}
    for doc in documents:
        sentence = doc.split()
        for word in sentence:
            if word not in idf_dict:  # Check to avoid recomputation
                idf_dict[word] = inverseDocumentFrequency(word, documents)
    return idf_dict

idf_dict = compute_idf(documents)

# 3.TF * IDF

Remember we are trying to find out relevant documents for the query: **life learning**

* For each term in the query multiply its normalized term frequency with its IDF on each document.
* In Document1 for the term life the normalized term frequency is 0.1 and its IDF is 1.405465108.
* Multiplying them together we get 0.140550715 (0.1 * 1.405465108).
*
Given below is TF * IDF calculations for life and learning in all the documents.

In [52]:
# tf-idf score across all docs for the query string("life learning")
def compute_tfidf_with_alldocs(documents, query):
    tf_idf = []
    query_tokens = query.split()
    df = pd.DataFrame(columns=['doc'] + query_tokens)
    for index, doc in enumerate(documents):
        df.loc[index, 'doc'] = index
        doc_num = tf_doc[index]
        sentence = doc.split()
        for word in sentence:
            if word in query_tokens:
                tf_idf_score = doc_num[word] * idf_dict[word]
                tf_idf.append(tf_idf_score)
                df.loc[index, word] = tf_idf_score
    df.fillna(0, inplace=True)
    return tf_idf, df

tf_idf, df = compute_tfidf_with_alldocs(documents, query)
print(df.head())

   doc      life  learning
0    0  0.000000  0.000000
1    1  0.002088  0.001348
2    2  0.010607  0.000000
3    3  0.000000  0.000000
4    4  0.000000  0.000000


# 4.Vector Space Models and Representation  – Cosine Similarity

The set of documents in a collection then is viewed as a set of vectors in a vector space. Each term will have its own axis. Using the formula given below we can find out the similarity between any two documents.

* > Cosine Similarity (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||
* > Dot product (d1,d2) = d1[0] * d2[0] + d1[1] * d2[1] * … * d1[n] * d2[n]
* > ||d1|| = square root(d1[0]2 + d1[1]2 + ... + d1[n]2)
* > ||d2|| = square root(d2[0]2 + d2[1]2 + ... + d2[n]2)


In [53]:
#from IPython.display import Image
#Image("../input/tfidf-kernel/cosinesimilarity.jpg")

* Vectors deals only with numbers. In this example we are dealing with text documents. This was the reason why we used TF and IDF to convert text into numbers so that it can be represented by a vecto


The query entered by the user can also be represented as a vector. We will calculate the TF*IDF for the query

In [54]:
# Compute term frequency for the query string
def compute_query_tf(query):
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word, query)
    return query_norm_tf

query_norm_tf = compute_query_tf(query1)

In [55]:
#tf-idf score for the query string("life learning")
def compute_query_tfidf(query):
    tfidf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tfidf_dict_qry[word] = query_norm_tf[word] * idf_dict_qry[word]
    return tfidf_dict_qry
tfidf_dict_qry = compute_query_tfidf(query1)
print(tfidf_dict_qry)

{'life': 1.0108256237659905, 'learning': 1.3047189562170503}


Let us now calculate the cosine similarity of the query and Document1.

Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

Dot product(Query, Document1)
     = ((0.702753576) * (0.140550715) + (0.702753576)*(0.140550715))
     = 0.197545035151

||Query|| = sqrt((0.702753576)2 + (0.702753576)2) = 0.993843638185

||Document1|| = sqrt((0.140550715)2 + (0.140550715)2) = 0.198768727354

Cosine Similarity(Query, Document) = 0.197545035151 / (0.993843638185) * (0.198768727354)
                                        = 0.197545035151 / 0.197545035151
                                        = 1

In [58]:
from collections.abc import Iterable
#Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

def cosine_similarity(tfidf_dict_qry, df, query, doc_num):
    """
    Calculate the cosine similarity between a query vector and a document vector represented in TF-IDF scores.

    Parameters:
    - tfidf_dict_qry: Dictionary of TF-IDF scores for the query.
    - df: DataFrame containing TF-IDF scores for all documents.
    - query: The query string.
    - doc_num: The index of the document to compare with the query.

    Returns:
    - cos_sim: Cosine similarity score.
    """
    dot_product = 0
    qry_mod = 0
    doc_mod = 0
    tokens = query.split()

    # Calculate dot product and vector magnitudes
    for keyword in tokens:
        if keyword in tfidf_dict_qry and keyword in df.columns:
            # Ensure the keyword exists in the DataFrame and query dictionary
            query_tfidf = tfidf_dict_qry.get(keyword, 0)
            doc_tfidf = df.loc[df['doc'] == doc_num, keyword].values[0] if keyword in df.columns else 0

            dot_product += query_tfidf * doc_tfidf
            qry_mod += query_tfidf ** 2
            doc_mod += doc_tfidf ** 2

    qry_mod = np.sqrt(qry_mod)
    doc_mod = np.sqrt(doc_mod)
    denominator = qry_mod * doc_mod

    cos_sim = dot_product / denominator if denominator != 0 else 0
    return cos_sim

def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable):
            for x in flatten(item):
                yield x
        else:
            yield item

In [63]:
def rank_similarity_docs(df, query_tf, query):
    cos_sim = []
    for doc_num in df.index:
        cos_sim.append(cosine_similarity(query_tf, df, query, doc_num))
    return cos_sim

similarity_scores = rank_similarity_docs(df, tfidf_dict_qry, query)

# Print the similarity scores
for doc_index, score in enumerate(similarity_scores):
    print(f"Document {doc_index + 1}: Cosine Similarity = {score:.4f}")

Document 1: Cosine Similarity = 0.0000
Document 2: Cosine Similarity = 0.9432
Document 3: Cosine Similarity = 0.6124
Document 4: Cosine Similarity = 0.0000
Document 5: Cosine Similarity = 0.0000
Document 6: Cosine Similarity = 0.7906
Document 7: Cosine Similarity = 0.7905
Document 8: Cosine Similarity = 0.6124
Document 9: Cosine Similarity = 0.0000
Document 10: Cosine Similarity = 0.7905
Document 11: Cosine Similarity = 0.0000
Document 12: Cosine Similarity = 1.0000
Document 13: Cosine Similarity = 0.0000
Document 14: Cosine Similarity = 0.6124
Document 15: Cosine Similarity = 0.0000
Document 16: Cosine Similarity = 0.0000
Document 17: Cosine Similarity = 0.0000
Document 18: Cosine Similarity = 0.6124
Document 19: Cosine Similarity = 0.6124
Document 20: Cosine Similarity = 0.6124
Document 21: Cosine Similarity = 0.0000
Document 22: Cosine Similarity = 0.0000
Document 23: Cosine Similarity = 0.0000
Document 24: Cosine Similarity = 0.0000
Document 25: Cosine Similarity = 0.0000


* I plotted vector values for the query and documents in 2-dimensional space of life and learning. Document1 has the highest score of 1. This is not surprising as it has both the terms life and learning.